In [11]:
import dicom
import os
import SimpleITK as sitk
import numpy as np
import array

import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interact

In [12]:
rp=dicom.read_file('ZORDAN/dicom/RP.201611829.PAROIG sansB.dcm')

In [13]:
iso=[]
for val in rp.BeamSequence[0].ControlPointSequence[0].IsocenterPosition: iso.append(float(val))
study=str(rp.PatientName.split('^')[0])
print study
#print iso


ZORDAN


In [14]:
def ct_viewer(im):
    def plot_image(myslice):
        plt.imshow(im[myslice], cmap=plt.cm.gray)
    interact(plot_image, myslice=(0, im.shape[0]))
    return; 
def ct_crop(pix, x1crop, x2cropt, y1crop, y2cropt):
    print 'ORIGINAL : '
    ct_viewer(pix)
    x2crop=x2cropt-pix.shape[2]
    y2crop=y2cropt-pix.shape[1]
    pix_crop=pixel_array[:, y1crop:y2crop, x1crop:x2crop]
    print 'CROPPED :'
    ct_viewer(pix_crop)
    return pix_crop;

In [15]:
#read dcm directory and compte ONE sitk 3D image object
reader=sitk.ImageSeriesReader()
#dicom_name=reader.GetGDCMSeriesFileNames('scanX/')
dicom_name=reader.GetGDCMSeriesFileNames(study +'/dicom/ct')
reader.SetFileNames(dicom_name)
ct_image = reader.Execute()

print ct_image.GetOrigin()
dicom_ct_dimension=ct_image.GetSize()
dicom_ct_pixel_size=ct_image.GetSpacing()
print'Dimesions : ' , dicom_ct_dimension
print'Pixels Size :' , dicom_ct_pixel_size

pixel_array=sitk.GetArrayFromImage(ct_image)   #Get numpy n-d array from sitk object to plot it                          
ct_viewer(pixel_array)

(-325.0, -325.0, -57.5)
Dimesions :  (512, 512, 128)
Pixels Size : (1.269531, 1.269531, 2.5)


interactive(children=(IntSlider(value=64, description=u'myslice', max=128), Output()), _dom_classes=('widget-interact',))

In [16]:
# write raw original ct
#pixel_array=pixel_array[::-1,::,:]
sitk.WriteImage(ct_image, study+'/images/'+study+'_original.mhd')

In [19]:
#select apropriates value to crop scan 
#use sliders to view all the slice and refer to original ct
x1=100  #   X axis lower value (on original ct)
x2=350  #   X axis upper value (on original ct)
y1=150 #   Y axis lower value (on original ct)
y2=370  #   Y axis upper value (on original ct)
pixel_array_crop=ct_crop(pixel_array, x1, x2, y1, y2);

ORIGINAL : 


interactive(children=(IntSlider(value=64, description=u'myslice', max=128), Output()), _dom_classes=('widget-interact',))

CROPPED :


interactive(children=(IntSlider(value=64, description=u'myslice', max=128), Output()), _dom_classes=('widget-interact',))

In [20]:
print pixel_array_crop.shape
#pixel_array_crop=pixel_array_crop[::-1,::,:]
print pixel_array_crop.shape

ct_viewer(pixel_array_crop)


ct_crop=sitk.GetImageFromArray(pixel_array_crop)
print ct_crop.GetSize()

crop_val=[x1, y1, 0]
new_origin=[]

for val1, cropval, space in zip(ct_image.GetOrigin(),crop_val , ct_image.GetSpacing()):
    new_origin.append(val1+(cropval*space))

ct_crop.SetOrigin(new_origin)
ct_crop.SetSpacing(ct_image.GetSpacing())


sitk.WriteImage(ct_crop, study+'/images/'+study+'.mhd')

(128, 220, 250)
(128, 220, 250)


interactive(children=(IntSlider(value=64, description=u'myslice', max=128), Output()), _dom_classes=('widget-interact',))

(250, 220, 128)
